# Web Scraping with Beautiful Soup

This tutorial will provide a very basic introduction to using the Beautiful Soup package to scrape text data from the web. 

## Installation

In a terminal, install Beautiful Soup if necessary by running <pre><code>conda install beautiful-soup</code></pre>

## Retrieving HTML

The general idea behind web scraping is to retrieve data that exists on a website, and convert it into a format that is usable for analysis. Webpages are rendered by the brower from HTML and CSS code, but much of the information included in the HTML underlying any website is not interesting to us.

We begin by reading in the source code for a given web page and creating a Beautiful Soup object with the BeautifulSoup function.

### urllib2
urllib2 is a module for working with urls, we will use it to open connections to urls and retrieve the webpage source.

In [2]:
from bs4 import BeautifulSoup
import urllib2
page = urllib2.urlopen('http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false').read()
soup = BeautifulSoup(page) # page is a string
print type(soup)

<class 'bs4.BeautifulSoup'>


The soup object contains all of the HTML in the original document.

In [3]:
print soup.prettify()[0:1000]

<!DOCTYPE html>
<html xmlns:fb="http://www.facebook.com/2008/fbml">
 <head>
  <script src="http://sports-ak.espn.go.com/sports/optimizely.js">
  </script>
  <meta charset="utf-8"/>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <link href="http://a.espncdn.com/prod/assets/icons/E.svg" mask="" rel="icon" sizes="any"/>
  <meta content="#CC0000" name="theme-color"/>
  <script type="text/javascript">
   if(true && navigator && navigator.userAgent.toLowerCase().indexOf("teamstream") >= 0) {
        window.location = 'http://a.m.espn.go.com/mobilecache/general/apps/sc';
    }
  </script>
  <title>
   2015-16 Regular Season NBA Player Stats and League Leaders - Scoring Per Game - National Basketball Association - ESPN
  </title>
  <meta content="xuj1ODRluWa0frM-BjIr_aSHoUC7HB5C1MgmYAM_GkA" name="google-site-verification"/>
  <meta content="B1FEB7C682C46C8FCDA3130F3D18AC28" name="msvalidate.01"/>
  <meta content="noodp" name="googlebot"/>
  <meta content="index, follow" na

In [4]:
#??soup.prettify()

## Parsing HTML

By "parsing HTML", we mean pulling out only the relevant tags/attributes for our analysis.  What Beautiful Soup does is provide a handy bunch of methods for doing this efficiently.

### find method

The find method will search for and return the first tag matching your corresponding search criteria, if it exists.  You can specify tag and attribute info etc.  There is also a findAll method that will return a collection of tags matching your query.

In [5]:
table_div = soup.find(id='my-players-table') # find first instance of the text, return that element.
print table_div.prettify()[0:4000]

<div class="col-main" id="my-players-table">
 <div class="mod-container mod-table">
  <div class="mod-header stathead">
   <h4>
    Points Per Game Leaders - All Players
   </h4>
  </div>
  <div class="mod-content">
   <table cellpadding="3" cellspacing="1" class="tablehead">
    <tr align="right" class="colhead">
     <td align="left" style="width:20px;">
      RK
     </td>
     <td align="left">
      PLAYER
     </td>
     <td align="left">
      TEAM
     </td>
     <td>
      <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/gamesPlayed/qualified/false" title="Games Played">
       GP
      </a>
     </td>
     <td>
      <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgMinutes/qualified/false" title="Minutes Per Game">
       MPG
      </a>
     </td>
     <td>
      <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/order/false" title="Points Per Game">
       PTS

Notice that the object returned by the find method is just another inner HTML structure, which we can step through just like we would have with the original soup object.  We've gone to the location in the webpage where the table that we seek starts, now we can use find again to get to the table data.

In [6]:
table = table_div.find("table")
print table

<table cellpadding="3" cellspacing="1" class="tablehead">
<tr align="right" class="colhead"><td align="left" style="width:20px;">RK</td><td align="left">PLAYER</td><td align="left">TEAM</td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/gamesPlayed/qualified/false" title="Games Played">GP</a></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgMinutes/qualified/false" title="Minutes Per Game">MPG</a></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/order/false" title="Points Per Game">PTS</a></td><td><span title="Field Goals Made-Attempted Per Game">FGM-FGA</span></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/fieldGoalPct/qualified/false" title="Field Goal Percentage">FG%</a></td><td><span title="3-Point Field Goals Made-Attempted Per Game">3PM-3PA</span></td><td><a href="http://espn.go.com/nba/statistics/play

### Searching by Attributes

Now that we have the table object, we need to step through the rows.  First we'll find the header row so we can populate what the field names will be in our data.  Here we're searching for tags under the table tag whose class attritbute is "colhead".  

In [7]:
# another way to use find: where class is "colhead" (See the html to see that it works.)

table_head = table.find(attrs={"class":'colhead'}) 
print table_head.prettify()

<tr align="right" class="colhead">
 <td align="left" style="width:20px;">
  RK
 </td>
 <td align="left">
  PLAYER
 </td>
 <td align="left">
  TEAM
 </td>
 <td>
  <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/gamesPlayed/qualified/false" title="Games Played">
   GP
  </a>
 </td>
 <td>
  <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgMinutes/qualified/false" title="Minutes Per Game">
   MPG
  </a>
 </td>
 <td>
  <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/order/false" title="Points Per Game">
   PTS
  </a>
 </td>
 <td>
  <span title="Field Goals Made-Attempted Per Game">
   FGM-FGA
  </span>
 </td>
 <td>
  <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/fieldGoalPct/qualified/false" title="Field Goal Percentage">
   FG%
  </a>
 </td>
 <td>
  <span title="3-Point Field Goals Made-Attempted Per Game">
   3PM-3PA
  </span>
 </td>
 <t

Now we find the actual values by searching for the 'td' tags, which is the tag for table data.

In [8]:
header_cols = table_head.findAll('td')
print header_cols

[<td align="left" style="width:20px;">RK</td>, <td align="left">PLAYER</td>, <td align="left">TEAM</td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/gamesPlayed/qualified/false" title="Games Played">GP</a></td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgMinutes/qualified/false" title="Minutes Per Game">MPG</a></td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/order/false" title="Points Per Game">PTS</a></td>, <td><span title="Field Goals Made-Attempted Per Game">FGM-FGA</span></td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/fieldGoalPct/qualified/false" title="Field Goal Percentage">FG%</a></td>, <td><span title="3-Point Field Goals Made-Attempted Per Game">3PM-3PA</span></td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/threePointFieldGoalPct/qualified/false" t

Finally, we step through these columns and save them to a list to be used later.  We'll ignore the rank column (RK) because that doesn't give us anything we want later.  We also separate the **PLAYER** column into **PLAYER** and **POSITION**.

In [9]:
cols = []
for header_col in header_cols:
    val = header_col.string
    if val != 'RK':
        cols.append(val)
    if val == 'PLAYER':
        cols.append('POSITION')
print cols

[u'PLAYER', 'POSITION', u'TEAM', u'GP', u'MPG', u'PTS', u'FGM-FGA', u'FG%', u'3PM-3PA', u'3P%', u'FTM-FTA', u'FT%']


### Stepping Through a Table

The table rows are indicated by the tag 'tr'.  Again we can find them all and iterate through them.  Within each row we iterate through the respective columns.

In [10]:
table_rows = table.findAll('tr')
print table_rows

[<tr align="right" class="colhead"><td align="left" style="width:20px;">RK</td><td align="left">PLAYER</td><td align="left">TEAM</td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/gamesPlayed/qualified/false" title="Games Played">GP</a></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgMinutes/qualified/false" title="Minutes Per Game">MPG</a></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/order/false" title="Points Per Game">PTS</a></td><td><span title="Field Goals Made-Attempted Per Game">FGM-FGA</span></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/fieldGoalPct/qualified/false" title="Field Goal Percentage">FG%</a></td><td><span title="3-Point Field Goals Made-Attempted Per Game">3PM-3PA</span></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/threePointFieldGoalPct/qu

We will save our results in 2 different ways to demonstrate how we can handle both.  The first will be a list of dicts where the key is the field name and the value is the field value.  The second will just be a list of lists of stats with no field name values (we've already defined them earlier).

In [11]:
players_stats_dicts = []
players_stats_array = []
for row in table_rows:
    if row.attrs['class'][0]=='colhead':
        continue
    player_stats = []
    row_cols = row.find_all('td')
    col_vals = []
    player_col = row_cols[1]
    player_name = player_col.find('a').string
    player_position = player_col.contents[1]
    player_position = player_position.split(' ')[1]
    player_stats.append(player_name)
    player_stats.append(player_position)
    for i in range(2, len(row_cols)):
        stat = row_cols[i].string
        player_stats.append(stat)
    players_stats_array.append(player_stats)
    player_stats = dict(zip(cols, player_stats))
    players_stats_dicts.append(player_stats)
print players_stats_dicts[0:5]        

[{u'FGM-FGA': u'11.1-20.1', u'MPG': u'33.3', u'FTM-FTA': u'6.1-6.7', u'FG%': u'.553', u'GP': u'7', u'PLAYER': u'Stephen Curry', u'FT%': u'.915', u'TEAM': u'GS', u'3PM-3PA': u'5.4-11.4', 'POSITION': u'PG', u'PTS': u'33.9', u'3P%': u'.475'}, {u'FGM-FGA': u'10.2-21.0', u'MPG': u'38.2', u'FTM-FTA': u'6.7-7.5', u'FG%': u'.484', u'GP': u'6', u'PLAYER': u'Kevin Durant', u'FT%': u'.889', u'TEAM': u'OKC', u'3PM-3PA': u'2.8-6.7', 'POSITION': u'SF', u'PTS': u'29.8', u'3P%': u'.425'}, {u'FGM-FGA': u'8.1-21.6', u'MPG': u'39.4', u'FTM-FTA': u'10.9-12.7', u'FG%': u'.377', u'GP': u'7', u'PLAYER': u'James Harden', u'FT%': u'.854', u'TEAM': u'HOU', u'3PM-3PA': u'2.6-10.7', 'POSITION': u'SG', u'PTS': u'29.7', u'3P%': u'.240'}, {u'FGM-FGA': u'11.7-19.5', u'MPG': u'35.2', u'FTM-FTA': u'6.0-7.7', u'FG%': u'.598', u'GP': u'6', u'PLAYER': u'Blake Griffin', u'FT%': u'.783', u'TEAM': u'LAC', u'3PM-3PA': u'0.0-0.5', 'POSITION': u'PF', u'PTS': u'29.3', u'3P%': u'.000'}, {u'FGM-FGA': u'10.0-21.3', u'MPG': u'36.7',

Here we've used the zip function to combine pairs of lists into tuples, and then transformed that into a dict to get a dictionary of FIELD --> VALUE for every player in the table.

Beautiful Soup has many other features, including the ability to step up, down, and sideways in the HTML tree and basically search for any tags, attributes, or values.  For more, take a look at the [Documentation](http://www.crummy.com/software/BeautifulSoup/bs4/doc/)

## Load into Pandas

Let's load our scraped data into Pandas and take a look at it.  Here is the first way we can do it, simply directly from the dictionary we defined.

In [12]:
import pandas as pd
import numpy as np

df = pd.DataFrame.from_dict(players_stats_dicts)
df.head()

# dictionaries do not have an inherent order.

,3P%,3PM-3PA,FG%,FGM-FGA,FT%,FTM-FTA,GP,MPG,PLAYER,POSITION,PTS,TEAM
0,.475,5.4-11.4,.553,11.1-20.1,.915,6.1-6.7,7,33.3,Stephen Curry,PG,33.9,GS
1,.425,2.8-6.7,.484,10.2-21.0,.889,6.7-7.5,6,38.2,Kevin Durant,SF,29.8,OKC
2,.240,2.6-10.7,.377,8.1-21.6,.854,10.9-12.7,7,39.4,James Harden,SG,29.7,HOU
3,.000,0.0-0.5,.598,11.7-19.5,.783,6.0-7.7,6,35.2,Blake Griffin,PF,29.3,LAC
4,.426,3.8-9.0,.469,10.0-21.3,.875,3.5-4.0,6,36.7,Damian Lillard,PG,27.3,POR


Now here is a 2nd way we can do it.  We convert the 2D stats array into a numpy array and create a Pandas dataframe from it along with the list of column headers we defined earlier.

In [13]:
np_array = np.array(players_stats_array) # create numpy array from matrix of statistics.
df = pd.DataFrame(np_array, columns=cols) # create dataframe from numpy array and name its columns.
df.head()

,PLAYER,POSITION,TEAM,GP,MPG,PTS,FGM-FGA,FG%,3PM-3PA,3P%,FTM-FTA,FT%
0,Stephen Curry,PG,GS,7,33.3,33.9,11.1-20.1,.553,5.4-11.4,.475,6.1-6.7,.915
1,Kevin Durant,SF,OKC,6,38.2,29.8,10.2-21.0,.484,2.8-6.7,.425,6.7-7.5,.889
2,James Harden,SG,HOU,7,39.4,29.7,8.1-21.6,.377,2.6-10.7,.240,10.9-12.7,.854
3,Blake Griffin,PF,LAC,6,35.2,29.3,11.7-19.5,.598,0.0-0.5,.000,6.0-7.7,.783
4,Damian Lillard,PG,POR,6,36.7,27.3,10.0-21.3,.469,3.8-9.0,.426,3.5-4.0,.875


## Exercise

The goal of this exercise is to combine the scoring and assists statistics for every player in the NBA in 2014-2015.  The end result will have them in a pandas dataframe with the fields from both pages for every player.

The general steps should be as follows:
- Create a function get_cols that retrieves the names of the header columns given a table element (skip the ranks, split the positions)
- Create a function get_data that retrieves the actual table data given a table element (skip the ranks, split the positions).  You can use either the dict approach or the numpy array approach.
- Write a python loop to loop through the various pages and call these functions on the appropriate urls so that you can retrieve every player (rather than just the top few).
- Repeat the above on both the scoring and assists URLs to get a pandas dataframe for both of them
- Use the pandas.DataFrame.join() function to join your 2 pandas dataframes together and get a total result

In [14]:
''' a function get_cols that retrieves the names of the header columns given a table element 
    (skip the ranks, split the positions)
'''
def get_cols(table):
    table_head = table.find(attrs={"class":'colhead'}) 
    #print table_head.prettify()
    header_cols = table_head.findAll('td')
    #print header_cols
    cols = []
    for header_col in header_cols:
        val = header_col.string
        if val != 'RK':
            cols.append(val)
        if val == 'PLAYER':
            cols.append('POSITION')
    return cols

In [15]:
''' a function get_data that retrieves the actual table data given a table element 
    (skip the ranks, split the positions). 
    I used the numpy array approach so that the columns would keep their ordering.
    The input  count  is the count of players already added from previous webpages.
'''
def get_data(table, count):
    table_rows = table.findAll('tr')
    #print table_rows
    players_stats_dicts = []
    players_stats_array = []
    for row in table_rows:
        #print row.prettify()
        if row.attrs['class'][0]=='colhead':
            continue
        player_stats = []
        row_cols = row.find_all('td')
        col_vals = []
        player_col = row_cols[1]
        player_name = player_col.find('a').string
        player_position = player_col.contents[1]
        player_position = player_position.split(' ')[1]
        player_stats.append(player_name)
        player_stats.append(player_position)
        for i in range(2, len(row_cols)):
            stat = row_cols[i].string
            player_stats.append(stat)
        players_stats_array.append(player_stats)
        #player_stats = dict(zip(cols, player_stats))
        #players_stats_dicts.append(player_stats)
        #print player_stats
    #print players_stats_dicts[-5:-1] 
    np_array = np.array(players_stats_array) # create numpy array from matrix of statistics.
    
    # create dataframe from numpy array and name its columns. Index it sequentially
    df = pd.DataFrame(np_array, columns=cols,index=range(count,len(np_array)+count)) 

    return df

In [16]:
''' The goal of this exercise is to combine the scoring and assists statistics 
    for every player in the NBA in 2014-2015. The end result will have them in a 
    pandas dataframe with the fields from both pages for every player.
'''

NBA_scoring_urlbase = 'http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/year/2015/qualified/false/count/'

''' Write a python loop to loop through the various pages and call these functions on the 
    appropriate urls so that you can retrieve every player (rather than just the top few).
'''

# initialize
cols = None
count = 0
first_iter = True

for n in range(1,482,40):
#for n in range(1,42,40):
    NBA_scoring_url = NBA_scoring_urlbase + str(n)
    
    # Find data on page exactly the same way we did above.
    NBA_scoring_page = urllib2.urlopen(NBA_scoring_url).read()
    soup = BeautifulSoup(NBA_scoring_page)
    table_div = soup.find_all(id='my-players-table') # find first instance of the text, return that element.
    if len(table_div) > 1:
        print("Warning: found multiple instances of 'my-players-table'." )
    table = table_div[0].find("table")
    
    # Scrape column headers
    if cols == None:
        cols = get_cols(table)
    else: # ensure that columns are the same on every subsequent page.
        if not cols == get_cols(table):
            print("Warning: unexpected columns on page " + str(n))

    # Scrape data
    current_df = get_data(table, count)
    
    # how many players have we added now?
    count = count + len(current_df)
    
    # join or merge dataframes from multiple pages
    if first_iter:
        NBAdf = current_df
        first_iter = False
    else:
        NBAdf = pd.concat([NBAdf,current_df])
    

In [20]:
NBAdf.tail()

,PLAYER,POSITION,TEAM,GP,MPG,PTS,FGM-FGA,FG%,3PM-3PA,3P%,FTM-FTA,FT%
487,Malcolm Lee,SG,PHI,1,2.0,0.0,0.0-1.0,.000,0.0-0.0,.000,0.0-0.0,.000
488,Kalin Lucas,PG,MEM,1,6.0,0.0,0.0-1.0,.000,0.0-0.0,.000,0.0-0.0,.000
489,Seth Curry,SG,PHX,2,4.0,0.0,0.0-1.5,.000,0.0-0.5,.000,0.0-0.0,.000
490,David Wear,PF,SAC,2,3.5,0.0,0.0-1.0,.000,0.0-0.5,.000,0.0-0.0,.000
491,Jerrelle Benimon,PF,UTAH,2,1.5,0.0,0.0-0.0,.000,0.0-0.0,.000,0.0-0.0,.000


In [21]:
NBAdf.describe()

,PLAYER,POSITION,TEAM,GP,MPG,PTS,FGM-FGA,FG%,3PM-3PA,3P%,FTM-FTA,FT%
count,492,492,492,492,492,492,492,492,492,492,492,492
unique,492,6,82,83,251,182,427,226,204,183,234,250
top,Paul Pierce,SG,LAL,82,18.9,2.7,1.3-3.5,.412,0.0-0.0,.000,0.0-0.0,.000
freq,1,112,17,27,6,10,4,9,97,112,18,19


In [23]:
''' Repeat the above on both the scoring and assists URLs to get a pandas dataframe for both of them
'''

# then combine assists and points pages for each player.
# for points, duplicate that for assists.
# assists: http://espn.go.com/nba/statistics/player/_/stat/assists/sort/avgAssists/year/2015/qualified/false/count/1
# look at join or merge in pandas.

NBA_assists_urlbase = 'http://espn.go.com/nba/statistics/player/_/stat/assists/sort/avgAssists/year/2015/qualified/false/count/'


# initialize
cols = None
count = 0
first_iter = True

for n in range(1,482,40):
#for n in range(1,42,40):
    NBA_assists_url = NBA_assists_urlbase + str(n)
    
    # Find data on page exactly the same way we did above.
    NBA_assists_page = urllib2.urlopen(NBA_assists_url).read()
    soup = BeautifulSoup(NBA_assists_page)
    table_div = soup.find_all(id='my-players-table') # find first instance of the text, return that element.
    if len(table_div) > 1:
        print("Warning: found multiple instances of 'my-players-table'." )
    table = table_div[0].find("table")
    
    # Scrape column headers
    if cols == None:
        cols = get_cols(table)
    else: # ensure that columns are the same on every subsequent page.
        if not cols == get_cols(table):
            print("Warning: unexpected columns on page " + str(n))

    # Scrape data
    current_df = get_data(table, count)
    
    # how many players have we added now?
    count = count + len(current_df)
    
    # join or merge dataframes from multiple pages
    if first_iter:
        NBA_assists_df = current_df
        first_iter = False
    else:
        NBA_assists_df = pd.concat([NBA_assists_df,current_df])

In [24]:
NBA_assists_df

,PLAYER,POSITION,TEAM,GP,MPG,AST,APG,TO,TOPG,AP48M,AST/TO
0,Chris Paul,PG,LAC,82,34.8,838,10.2,190,2.3,14.1,4.41
1,John Wall,PG,WSH,79,35.9,792,10.0,304,3.8,13.4,2.61
2,Ty Lawson,PG,DEN,75,35.5,720,9.6,185,2.5,13.0,3.89
3,Ricky Rubio,PG,MIN,22,31.5,193,8.8,64,2.9,13.4,3.02
4,Russell Westbrook,PG,OKC,67,34.4,574,8.6,293,4.4,12.0,1.96
5,Rajon Rondo,PG,BOS/DAL,68,29.7,538,7.9,210,3.1,12.8,2.56
6,Stephen Curry,PG,GS,80,32.7,619,7.7,249,3.1,11.4,2.49
7,LeBron James,SF,CLE,69,36.1,511,7.4,272,3.9,9.8,1.88
8,Jeff Teague,PG,ATL,73,30.5,513,7.0,205,2.8,11.1,2.50
9,James Harden,SG,HOU,81,36.8,565,7.0,321,4.0,9.1,1.76
